# Librerias

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama.llms import OllamaLLM
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document

# Lectura de pdfs

In [ ]:
# Función para cargar y procesar PDFs
"""
Procesa una lista de PDFs, los divide en fragmentos y crea un vector store con FAISS.
"""
# Cargar documentos desde los PDFs

pdf_paths = ['naturaleza-yo.pdf']
documents = []

for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path)
    documents.extend(loader.load())

# Procesado de datos en vectores

In [ ]:
# Dividir texto en fragmentos manejables
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

# Crear embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")

# Extraer texto de los documentos divididos
texts = [doc.page_content for doc in split_docs]

# Crear el vector store
vector_store = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    persist_directory="./chroma_pdf_facts",
)

retriever = vector_store.as_retriever()

# Modelo de lenguaje (LLM)

In [4]:
"""
Realiza una consulta en el vector store y utiliza un modelo LLM para generar una respuesta.
"""
# Prompt
template = """Responde en base al siguiente contexto:
{context}

Pregunta: {question}
Respuesta: en español.
"""
prompt = ChatPromptTemplate.from_template(template)

# Crear un LLM de OpenAI
llm = OllamaLLM(model='llama3.2')

def retrieve_context(query):
    # Recuperar fragmentos relevantes del vector store
    results = retriever.get_relevant_documents(query)
    # Combinar los textos de los documentos recuperados
    context = "\n".join([doc.page_content for doc in results])
    return context

# Chain
chain = (
    {"question": RunnablePassthrough()}
    | {"context": retrieve_context} 
    | prompt
    | llm
    | StrOutputParser()
)


# Main Code

In [6]:
# Ejemplo de uso
if __name__ == "__main__":
    
    # Realizar una consulta
    pregunta = "Qué les pasó a Nacho y a María?"
    print(f'Pregunta: {pregunta}')
    print("Consultando el sistema RAG...")
    # Ejecutar la consulta

    print("Respuesta:")
    print(chain.invoke(pregunta))


Pregunta: Qué les pasó a Nacho y a María?
Consultando el sistema RAG...
Respuesta:
Nacho y María sufrieron un diluvio. La lluvia era muy fuerte y se colaba por las paredes de su casa de palma de coco, lo que hizo que el candil que había encendido María se moviera hacia todos lados. El relámpago y la débil luz del candil dejaban ver en la esquina de la casa; el rostro asustado de Manuel, el hijo de María y Nacho, quien tenía nueve años.
